In [1]:
pip install h5netcdf 

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install xarray

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install cftime

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install toolz

Note: you may need to restart the kernel to use updated packages.


In [5]:
import xarray as xr
import numpy as np
import datetime
import pandas as pd
import cftime

### **<font color="Purple"> Datos de ERA5 </font>**

### **<font color="olive"> CAOs </font>**

In [6]:
fechasCAOE5 = pd.read_csv('datasetsok/fechasCAO_E5.csv')
# fechasCAOE5

### **<font color="olive"> Geopotencial</font>**

In [7]:
csv_df = pd.read_csv('datasets/PNAindexcsv.csv')
# csv_df.head()

In [8]:
csv_df=csv_df.rename({'pna_index_cdas':'PNAidx'},axis=1)
csv_df["fecha"] = pd.to_datetime(csv_df[["year", "month", "day"]])
df79_24 = csv_df.loc[csv_df.year.between(1979, 2024)]

### **<font color="darkpink"> Concat de PNA y CAOs </font>**

Tengo que elegir la columna de fecha para relacionarla con CAOs, después ligarla con la de PNAidx para ver si eran positivos o negativos, o neutros.

In [9]:
def PNA_CAOsE5(df_caos, df_pna,thr):
    """
    Clasifica las fechas de CAO según la fase del índice PNA.

    Parámetros:
    -----------
    df_caos : DataFrame
        DataFrame con una columna 'fecha' que contiene las fechas de los eventos CAO.
    
    df_pna : DataFrame
        DataFrame con columnas 'fecha' y 'PNAidx', donde 'PNAidx' es el valor diario del índice PNA.

    Retorna:
    --------
    dict : Diccionario con:
        - 'PNAyCAOS': DataFrame con fechas de CAO y su valor PNA correspondiente
        - 'PNApCAO': Lista de dicts con fechas de CAO en fase positiva del PNA
        - 'PNAnCAO': Lista de dicts con fechas de CAO en fase negativa del PNA
        - 'PNA0CAO': Lista de dicts con fechas de CAO en fase neutra del PNA
    """
    # Asegurar que las fechas estén en formato datetime y sin hora
    df_caos = df_caos.copy()
    df_pna = df_pna.copy()
    
    df_caos['fecha'] = pd.to_datetime(df_caos['fecha']).dt.floor('D')
    df_pna['fecha'] = pd.to_datetime(df_pna['fecha']).dt.floor('D')
    
    # Unir los DataFrames por fecha
    caos_pna = df_caos.merge(df_pna, on='fecha', how='inner')

    # Diccionarios vacíos para fechas donde PNA es +,-,0
    PNA_p_CAO = []
    PNA_n_CAO = []
    PNA_0_CAO = []

    # Clasificar según el índice PNA
    for _, row in caos_pna.iterrows():
        fecha = row['fecha']
        PNAidx = row['PNAidx']

        if PNAidx >= thr:
            PNA_p_CAO.append({'fecha': fecha, 'PNAidx': PNAidx})
        elif PNAidx <= -thr:
            PNA_n_CAO.append({'fecha': fecha, 'PNAidx': PNAidx})
        elif -thr <= PNAidx <= thr:
            PNA_0_CAO.append({'fecha': fecha, 'PNAidx': PNAidx})

    return {
        'PNAyCAOS': caos_pna,
        'PNApCAO': PNA_p_CAO,
        'PNAnCAO': PNA_n_CAO,
        'PNA0CAO': PNA_0_CAO
           }


In [18]:
df79_24['PNAidx'].std()

0.8434023258015263

In [19]:
PNAconcatCAO_E5 = PNA_CAOsE5(fechasCAOE5, df79_24,thr=float(df79_24['PNAidx'].std())/2)

##### **<font color="darkpink"> ERA5 </font>**

In [20]:
print(f'Cantidad de CAOs en ERA5: {len(PNAconcatCAO_E5["PNAyCAOS"])}')
print(f'Cantidad de CAOs en fase positiva: {len(PNAconcatCAO_E5["PNApCAO"])}')
print(f'Cantidad de CAOs en fase negativa: {len(PNAconcatCAO_E5["PNAnCAO"])}')
print(f'Cantidad de CAOs en fase neutra: {len(PNAconcatCAO_E5["PNA0CAO"])}')

Cantidad de CAOs en ERA5: 70
Cantidad de CAOs en fase positiva: 50
Cantidad de CAOs en fase negativa: 6
Cantidad de CAOs en fase neutra: 14
